# Step1. Setup paths

In [1]:
import os
os.chdir('/home/mic/Desktop/Udacity 2020/data engineering/Projects Repo/data_engineering_udacity/Proj2 AWS DWH/')

# Step2. Setup AWS

In [2]:
from functions.setup_aws import *

### Step 2a. Setup Redshift cluster

In [3]:
#create a client for Redshift
redshift_admin = create_redshift_admin('configs/iam.cfg')

In [ ]:
#create a Redshift cluster with specs defined in dwh.cfg
response = create_cluster('configs/dwh.cfg', redshift_admin)

In [4]:
#get cluster status
#cluster_id = response['Cluster']['ClusterIdentifier']
cluster_id = 'dwhcluster6'

cluster_stats = redshift_admin.describe_clusters(ClusterIdentifier = cluster_id)['Clusters'][0]
#cluster_stats

### Step2b. Open endpoint for inbound connections

In [5]:
#create ec2 instance
ec2 = create_ec2_instance('configs/iam.cfg')

In [ ]:
#open an incoming TCP port to access the cluster ednpoint
open_tcp_endpoint(ec2, cluster_stats)

In [6]:
#test connection
conn = make_connection('configs/dwh.cfg', cluster_stats)
print(conn)

<connection object at 0x7f01c3f58c20; dsn: 'user=dwhuser password=xxx dbname=dwh host=dwhcluster6.c4kzd5f8jllk.eu-west-1.redshift.amazonaws.com port=5439', closed: 0>


# Clean-up

In [ ]:
#delete cluster
delete_cluster(cluster_id, redshift_admin)

In [ ]:
#verify cluster deletion
try:
    status = redshift_admin.describe_clusters(ClusterIdentifier = cluster_id)['Clusters'][0]['ClusterStatus']
    print('Cluster status: {}'.format(status))
except Exception as e:
    print(e)